# Useful functions 

In [1]:
# Lists full paths of all items in directory, d
def listdir_fullpath(d):
    return [os.path.join(d, f) for f in os.listdir(d)]

In [2]:
# Reorders the tifs (i.e. individual bands) downloaded from GEE according to feature order in feat_list_new,
# then stacks them all into one multiband image called 'stack.tif' located in input path
# Reqs: rasterio, os, from zipfile import *
def tifStacker(path, feat_list_new): 

    file_list = []
    
    # This gets the name of all files in the zip folder, and formats them into a full path readable by rasterio.open()
    with ZipFile(path, 'r') as f:
        names = f.namelist()
        names = ['zip://'+path+'!'+name for name in names]
        for file in names:
            if file.endswith('.tif'):
                file_list.append(file)
    
    feat_list_files = list(map(lambda x: x.split('.')[-2], file_list)) # Grabs a list of features in file order        
    
    # Create 1 row df of file names where each col is a feature name, in the order files are stored locally
    file_arr = pd.DataFrame(data=[file_list], columns=feat_list_files)

    # Then index the file list by the ordered list of feature names used in training
    file_arr = file_arr.loc[:, feat_list_new]

    # The take this re-ordered row as a list - the new file_list
    file_list = list(file_arr.iloc[0,:])
    
    # Read metadata of first file. This needs to be a band in float32 dtype, because it sets the metadata for the entire stack
    # and we are converting the other bands to float64
    with rasterio.open(file_list[1]) as src0:
        meta = src0.meta
        meta['dtype'] = 'float32'
    #         print(meta)

    # Update meta to reflect the number of layers
    meta.update(count = len(file_list))

    # Read each layer, convert to float, and write it to stack
    # There's also a gdal way to do this, but unsure how to convert to float: https://gis.stackexchange.com/questions/223910/using-rasterio-or-gdal-to-stack-multiple-bands-without-using-subprocess-commands

    # Make new directory for stacked tif if it doesn't already exist
    try:
        os.mkdir(path + '/../'+'/stack')
    except FileExistsError:
        print('Stack directory already exists')

    # Remove stack file if already exists
    try:
        os.remove(path + '/../' + '/stack/stack.tif')
        print('Removing existing "stack.tif" and creating new one')
    except FileNotFoundError:
    #     pass
        print('Creating "stack.tif"')

    with rasterio.open(path + '/stack/stack.tif', 'w', **meta) as dst:
        for id, layer in enumerate(file_list, start=0):
            with rasterio.open(layer) as src1:
                dst.write_band(id+1, src1.read(1).astype('float32'))